# Data Source
https://docs.google.com/spreadsheets/d/1irg60f9qsZOkhp0cwOU7Cy4rJQeyusEUzTNQzhoTYTU/edit?gid=0#gid=0

Let's sanitize this into a nice JSON file to use as a data source for the website.


In [1]:
import { DeviceParser } from "./device.parser.ts";

// parse handhelds html 
const parsedDevices = DeviceParser.parseHandheldsHtml("./original-handhelds.html");
const devices = parsedDevices.filter((device) => device.name.sanitized !== "");


In [2]:
// marshall into JSON and into bytes
const devicesJson = JSON.stringify(devices, null, 2);
const devicesJsonBytes = new TextEncoder().encode(devicesJson);

In [3]:
// place source/handhelds.json
const filePath = `results/handhelds.json`;
await Deno.mkdir('results', { recursive: true });
await Deno.writeFile(filePath, devicesJsonBytes);
